In [1]:
import sys
from pathlib import Path
import os
import json

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\ModelDevelopment\experiments


In [2]:
gt_test = DataPaths.TEST_DATA_GT.value
#Schema of gt_test: {"0": 4, "1": 93, "2": 25, "3": -1, "4": -1,

# load the json and compile a list of all tracklets with -1 as not_legible and all other tracklets as legible
with open(gt_test, 'r') as f:
    data = json.load(f)
    legible = []
    illegible = []
    
    for key, value in data.items():
        if value == -1:
            illegible.append(key)
        else:
            legible.append(key)
    
print("Legible tracklets: ", legible)
print("Illegible tracklets: ", illegible)

Legible tracklets:  ['0', '1', '2', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '26', '27', '28', '30', '31', '32', '33', '34', '35', '36', '37', '42', '43', '44', '45', '46', '48', '52', '54', '55', '56', '57', '58', '59', '60', '62', '63', '64', '65', '66', '67', '68', '69', '70', '72', '73', '77', '80', '83', '84', '86', '88', '91', '92', '93', '94', '95', '96', '98', '99', '100', '101', '102', '103', '105', '106', '107', '109', '111', '112', '113', '114', '115', '116', '120', '123', '125', '126', '127', '128', '131', '132', '133', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '148', '149', '150', '151', '152', '153', '154', '156', '157', '159', '160', '161', '163', '166', '169', '170', '171', '173', '178', '179', '180', '181', '185', '186', '187', '188', '189', '190', '191', '192', '193', '196', '197', '198', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '212', '213', '214', '219'

In [3]:
# # Delete soccer_ball.json files
# import os
# root_path = r"C:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images"

# for tracklet in range(1, 1210):
#   path = os.path.join(root_path, str(tracklet))
  
#   # If there is a .soccer_ball.json file in there, delete it
#   if os.path.exists(os.path.join(path, "soccer_ball.json")):
#     os.remove(os.path.join(path, "soccer_ball.json"))

In [4]:
pipeline = CentralPipeline(
  #tracklets_to_process_override=[109, 88, 34, 151, 19, 68, 67, 128, 57, 178, 189, 196, 214, 242, 258, 271, 304, 327, 320, 352, 375, 378, 414, 416, 439, 425, 451, 453, 454, 466, 499, 511, 513, 526, 528, 567, 576, 594, 619, 623, 646, 687, 682, 709, 711, 717, 739, 758, 767, 795, 788, 840, 843, 832, 841, 852, 847, 851, 856, 850, 868, 866, 871, 901, 902, 904, 900, 907, 928, 939, 944, 961, 984, 983, 1005, 1039, 1064, 1063, 1077, 1091, 1101, 1137, 1152, 1153, 1160, 1167, 1157, 1165, 1206],
  #tracklets_to_process_override=[327, 711, 907, 1064], # These ones are now passing 
  #tracklets_to_process_override=illegible,
  tracklets_to_process_override=[168, 167, 162, 97, 39, 87, 89, 134, 61, 172, 164, 199, 215, 260, 267, 269, 281, 276, 288, 295, 317, 325, 340, 345, 374, 394, 388, 386, 391, 406, 408, 434, 437, 459, 471, 514, 532, 558, 683, 748, 770, 790, 802, 827, 861, 869, 879, 897, 895, 909, 964, 981, 999, 1023, 1031, 1025, 1033, 1030, 1055, 1088, 1087, 1169, 1175, 1174, 1173, 1183, 1170, 1179, 1184, 1185],
  #num_tracklets=1,
  #num_images_per_tracklet=50,
  input_data_path=DataPaths.TEST_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value,
  display_transformed_image_sample=False, # NOTE: DO NOT USE. Code is parallelized so we cannot show images anymore. Code breaks, but first one will show if True.
  num_image_samples=1,
  use_cache=False, # Set to false to rebuild the cache
  suppress_logging=False,
  
  # --- PARALLELIZATION PARAMS --- These settings are optimal for an NVIDIA RTX 3070 Ti Laptop GPU.
  num_workers=4,            # CRITICAL optimisation param. Adjust accordingly. 62
  tracklet_batch_size=32,   # CRITICAL optimisation param. Adjust accordingly. 32
  image_batch_size=100,     # CRITICAL optimisation param. Adjust accordingly. 200
  num_threads_multiplier=4  # CRITICAL optimisation param. Adjust accordingly. 3. Interpretation: num_threads = num_workers * num_threads_multiplier
  )

2025-03-29 23:06:48 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-03-29 23:06:48 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-03-29 23:06:48 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-03-29 23:06:48 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-03-29 23:06:48 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-03-29 23:06:48 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-03-29 23:06:48 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
2

In [5]:
pipeline.run_soccernet(
  run_soccer_ball_filter=True,
  generate_features=True,
  run_filter=True,
  run_legible=True,
  run_legible_eval=True,
  run_pose=False, ##
  run_crops=False,
  run_str=False,
  run_combine=False,
  run_eval=False)

2025-03-29 23:06:48 [INFO] Running the SoccerNet pipeline.
2025-03-29 23:06:48 [INFO] Tracklet override applied. Using provided tracklets: 168, 167, 162, 97, 39, 87, 89, 134, 61, 172, 164, 199, 215, 260, 267, 269, 281, 276, 288, 295, 317, 325, 340, 345, 374, 394, 388, 386, 391, 406, 408, 434, 437, 459, 471, 514, 532, 558, 683, 748, 770, 790, 802, 827, 861, 869, 879, 897, 895, 909, 964, 981, 999, 1023, 1031, 1025, 1033, 1030, 1055, 1088, 1087, 1169, 1175, 1174, 1173, 1183, 1170, 1179, 1184, 1185


  0%|          | 0/3 [00:00<?, ?it/s]

2025-03-29 23:06:48 [INFO] Tracklet batch size: 32
2025-03-29 23:06:48 [INFO] Image batch size: 100
2025-03-29 23:06:48 [INFO] Number of workers: 4
2025-03-29 23:06:48 [INFO] Number of threads created: 16
2025-03-29 23:06:48 [INFO] Using double parallelization: multiprocessing + CUDA batch processing.


Processing tracklets (CUDA + CPU):   0%|          | 0/32 [00:00<?, ?it/s]

2025-03-29 23:07:13 [INFO] Creating placeholder data files for Soccer Ball Filter.
2025-03-29 23:07:14 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\168\features.npy
2025-03-29 23:07:14 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\167\features.npy
2025-03-29 23:07:14 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\162\features.npy
2025-03-29 23:07:14 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\97\features.npy
2025-03-29 23:07:14 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\c

...

2025-03-29 23:14:55 [INFO] Saved features for tracklet with shape (558, 2048) to c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\1170\features.npy
2025-03-29 23:14:55 [INFO] Identifying and removing outliers by calling gaussian_outliers.py on feature file


Lightning automatically upgraded your loaded checkpoint from v1.1.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\pre_trained_models\reid\dukemtmcreid_resnet50_256_128_epoch_120.ckpt`


2025-03-29 23:14:58 [INFO] Saved features for tracklet with shape (615, 2048) to c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\1183\features.npy
2025-03-29 23:14:58 [INFO] Identifying and removing outliers by calling gaussian_outliers.py on feature file
2025-03-29 23:14:58 [INFO] 2025-03-29 23:14:57 [INFO] Threshold for scores: 0.999, rounds: 3
2025-03-29 23:14:57 [INFO] Round 1: Excluded 1 outliers
2025-03-29 23:14:57 [INFO] Round 1: Average score: 0.9991579055786133
2025-03-29 23:14:57 [INFO] Excluded images: ['1173_99.jpg']
2025-03-29 23:14:57 [INFO] Round 2: Excluded 1 outliers
2025-03-29 23:14:57 [INFO] Round 2: Average score: 0.999954104423523
2025-03-29 23:14:57 [INFO] Excluded images: ['1173_99.jpg']
2025-03-29 23:14:57 [INFO] Round 3: Excluded 1 outliers
2025-03-29 23:14:57 [INFO] Round 3: Average score: 0.999954104423523
2025-03-29 23:14:57 [INFO] Excluded images: ['1173_99.jpg']

2025-03-29 23:14:58 [ERROR] 
2025-03-

Evaluating legibility:   0%|          | 0/70 [00:00<?, ?it/s]

2025-03-29 23:15:44 [INFO] Correct 18 out of 70. Accuracy 25.714285714285715%.
2025-03-29 23:15:44 [INFO] TP=0, TN=18, FP=52, FN=0
2025-03-29 23:15:44 [INFO] Precision=0.0, Recall=0
2025-03-29 23:15:44 [INFO] F1=0
2025-03-29 23:15:44 [INFO] Saved failed legibility cases to: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\failed_legibility_cases.json


## Benchmark (current pipeline metrics)
### Results
- 2025-03-29 14:44:05 [INFO] Correct 1051 out of 1210. Accuracy 86.85950413223141%.
- 2025-03-29 14:44:05 [INFO] TP=766, TN=285, FP=70, FN=89
- 2025-03-29 14:44:05 [INFO] Precision=0.916267942583732, Recall=0.895906432748538
- 2025-03-29 14:44:05 [INFO] F1=0.9059727971614429
- 2025-03-29 14:44:05 [INFO] Saved failed legibility cases to:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\failed_legibility_cases.json

### Analysis
legible_but_marked_illegible: 89 cases
109, 88, 34, 151, 19, 68, 67, 128, 57, 178, 189, 196, 214, 242, 258, 271, 304, 327, 320, 352, 375, 378, 414, 416, 439, 425, 451, 453, 454, 466, 499, 511, 513, 526, 528, 567, 576, 594, 619, 623, 646, 687, 682, 709, 711, 717, 739, 758, 767, 795, 788, 840, 843, 832, 841, 852, 847, 851, 856, 850, 868, 866, 871, 901, 902, 904, 900, 907, 928, 939, 944, 961, 984, 983, 1005, 1039, 1064, 1063, 1077, 1091, 1101, 1137, 1152, 1153, 1160, 1167, 1157, 1165, 1206

illegible_but_marked_legible: 70 cases
168, 167, 162, 97, 39, 87, 89, 134, 61, 172, 164, 199, 215, 260, 267, 269, 281, 276, 288, 295, 317, 325, 340, 345, 374, 394, 388, 386, 391, 406, 408, 434, 437, 459, 471, 514, 532, 558, 683, 748, 770, 790, 802, 827, 861, 869, 879, 897, 895, 909, 964, 981, 999, 1023, 1031, 1025, 1033, 1030, 1055, 1088, 1087, 1169, 1175, 1174, 1173, 1183, 1170, 1179, 1184, 1185

- Total misclassifications: 159
- False negatives (legible marked as illegible): 89
- False positives (illegible marked as legible): 70
- Percentage of false negatives: 7.36%
- Percentage of false positives: 5.79%
- Combined percentage of errors as a percentage of total cases: 13.14%

## New findings
Tracklets that were all marked as illegible with the old one but should be legible (want to see anything > 0% accuracy here)
- 2025-03-29 23:00:00 [INFO] Correct 64 out of 89. Accuracy 71.91011235955057%.
- 2025-03-29 23:00:00 [INFO] TP=64, TN=0, FP=0, FN=25
- 2025-03-29 23:00:00 [INFO] Precision=1.0, Recall=0.7191011235955056
- 2025-03-29 23:00:00 [INFO] F1=0.8366013071895425

Tracklets that were all marked as legible with the old one but should be illegible (want to see anything > 0% accuracy here)
- 2025-03-29 23:15:44 [INFO] Correct 18 out of 70. Accuracy 25.714285714285715%.
- 2025-03-29 23:15:44 [INFO] TP=0, TN=18, FP=52, FN=0
- 2025-03-29 23:15:44 [INFO] Precision=0.0, Recall=0
- 2025-03-29 23:15:44 [INFO] F1=0

In [6]:
import json
failed_cases = f"{DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value}/failed_legibility_cases.json"

# Load this JSON file
with open(failed_cases, 'r') as f:
    failed_cases_dict = json.load(f)
    # Print the every key and its associated value (value will be a list so need to make that printable with ",".join):
    for key, value in failed_cases_dict.items():
        print(f"{key}: {len(value)} cases")
        if len(value) > 0:
            print(f"{', '.join(value)}")
        print()
    
    # Summary statistics
    legible_but_marked_illegible = len(failed_cases_dict.get("legible_but_marked_illegible", []))
    illegible_but_marked_legible = len(failed_cases_dict.get("illegible_but_marked_legible", []))
    total_errors = legible_but_marked_illegible + illegible_but_marked_legible
    
    print(f"Total misclassifications: {total_errors}")
    print(f"False negatives (legible marked as illegible): {legible_but_marked_illegible}")
    print(f"False positives (illegible marked as legible): {illegible_but_marked_legible}")
    print(f"Percentage of false negatives: {legible_but_marked_illegible / 1210 * 100:.2f}%")
    print(f"Percentage of false positives: {illegible_but_marked_legible / 1210 * 100:.2f}%")
    print(f"Combined percentage of errors as a percentage of total cases: {total_errors / 1210 * 100:.2f}%")

legible_but_marked_illegible: 0 cases

illegible_but_marked_legible: 52 cases
1170, 999, 1185, 1023, 406, 386, 61, 1031, 408, 288, 1025, 861, 172, 1033, 295, 164, 317, 459, 325, 199, 1030, 391, 471, 162, 89, 981, 1088, 340, 215, 1055, 260, 168, 1169, 879, 267, 558, 269, 897, 87, 683, 394, 97, 1173, 909, 748, 281, 39, 532, 964, 1183, 1175, 1179

Total misclassifications: 52
False negatives (legible marked as illegible): 0
False positives (illegible marked as legible): 52
Percentage of false negatives: 0.00%
Percentage of false positives: 4.30%
Combined percentage of errors as a percentage of total cases: 4.30%


In [7]:
# import timm
# import torch
# from torch import nn

# model = timm.create_model("timm/vit_base_patch16_224.orig_in21k_ft_in1k", pretrained=False)
# model.head = nn.Linear(model.head.in_features, 10)
# model.load_state_dict(
#     torch.hub.load_state_dict_from_url(
#         "https://huggingface.co/edadaltocg/vit_base_patch16_224_in21k_ft_svhn/resolve/main/pytorch_model.bin",
#         map_location="cpu",
#         file_name="vit_base_patch16_224_in21k_ft_svhn.pth",
#     )
# )